# Example 6: Functions with integer covariates

In this advanced example we show how to solve optimization problems where one or all covariates are integers, including how to modulate the number of random datapoints to improve convergence. We will also show how to configure the data types of the covariates and set the covariate names.

We will start by keeping both covariates as integers and will towards the end replace one of the covariates with a continuous variable.


## Problem solved in this notebook

In this notebook we will illustrate how to use relative improvement by optimizing this known function 

$$
f( x_0 , x_1 ) = \sin{ \left( \frac{x_0 \pi}{ 10 } \right)} \left[ - \left(6 \frac{x_1}{ 10 } - 2 \right)^2 \sin{ \left(12 \frac{x_1}{ 10 } - 4 \right)} \right], \quad x_0 , x_1 \in \mathcal{Z}^{+}_{0;10}.
$$

It is known that the function above has its maximum at $(x_0, x_1)^* = (5,8)$ with a corresponding response of $f(x_0^*, x_1^*) = 4.94913$.

We will solve it using the `.auto`-method in and apply conditions on the relative improvement and illustrate their impact on covergence for two different cases

1. Using the default settings
2. Tweak the rate of random sampling in between Bayesian optimization steps to improve convergence.
3. Convert $x_1$ to a continuous variable and solve the problem.

In [ ]:
! pip install --user greattunes

# import
import numpy as np
import pandas as pd
import torch
from greattunes import CreativeProject
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

## Define the function to optimize

In [ ]:
# scale the covariates
x0scale = 10
x1scale = 10

# the function to optimize
# takes a pandas dataframe as input and returns an array
def f(x):
    x0, x1 = np.meshgrid(x["x0"].values, x["x1"].values)
    x0p = np.round(x0,0)/x0scale
    x1p = np.round(x1,0)/x1scale
    return np.sin(x0p*np.pi)*(-(6*x1p - 2)**2*np.sin(12*x1p-4))

Plot the function $f$

In [ ]:
# helper for plotting
def f_plot(x0vec,x1vec):
    xdf = pd.DataFrame({"x0": x0vec, "x1": x1vec})
    x0p, x1p = np.meshgrid(x0vec, x1vec)
    return x0p, x1p, f(xdf)
    
# generate the data to plot
x0vec = np.linspace(0,x0scale,200)
x1vec = np.linspace(0,x1scale,200)

x0p, x1p, output = f_plot(x0vec, x1vec)

# Set up a figure twice as tall as it is wide
fig = plt.figure(figsize=(12,6)) 

# First subplot: contour plot 
ax = fig.add_subplot(1, 2, 1)
cs = ax.contourf(x0p, x1p, output, cmap="jet")
ax.set_xlabel("$x_0$")
ax.set_ylabel("$x_1$")
cbar = fig.colorbar(cs)


# Second subplot: surface plot
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.view_init(elev=20., azim=19)
ax.plot_surface(x0p, x1p, output, rstride=1, cstride=1,linewidth=1, antialiased=True, shade=True, cmap="jet")
ax.set_xlabel("$x_0$")
ax.set_ylabel("$x_1$")
ax.set_zlabel("Response")

plt.show()

Define the covariate variables and name them

In [ ]:
# define the range of interest
x0_init = 2
x1_init = 6

In [ ]:
# covariate names
x0_name = "x0"
x1_name = "x1"

# set data type for covariates. For integer covariates use type int
x0_type = int
x1_type = int

# create the covariate-defining data structure as a named nested dicts
covars2d = {
    x0_name: {
        "guess": x0_init,
        "min": 0,
        "max": x0scale,
        "type": int,
    },
    x1_name: {
        "guess": x1_init,
        "min": 0,
        "max": x1scale,
        "type": int,
    },
}

## 1: Solve the problem with standard settings

In [ ]:
# initialize class instance
cc = CreativeProject(covars=covars2d)

# number of iterations
max_iter = 90

# run the auto-method
cc.auto(response_samp_func=f, max_iter=max_iter)

Show the results

In [ ]:
# run current_best method
cc.current_best()

In [ ]:
# illustrate the convergence
cc.plot_convergence()

In [ ]:
# plot best result vs iterations
cc.plot_best_objective()

## 2: Improve convergence by increasing random sampling

Here we will start by running first 6 random samples using latin hypercube sampling. We set this via the`num_initiaL_random` argument. Secondly we will also increase the cadence by which random samples will be interdispersed among the Bayesian steps by setting `random_step_candence` to 5 instead of the 10 used by default. Beware that it is also possible to change between different sampling methods (current fully random and latin hypercube sampling methods are available).

In [ ]:
# initialize class instance
cc2 = CreativeProject(covars=covars2d, random_step_cadence=5, num_initial_random=6)

# number of iterations
max_iter = 90

# run the auto-method
cc2.auto(response_samp_func=f, max_iter=max_iter)

In [ ]:
# run current_best method
cc2.current_best()

In [ ]:
# plot convergence
cc2.plot_convergence()

In [ ]:
# plot evolution of best result of objective function
cc2.plot_best_objective()

## 3: Converting covariate $x_1$ to a continuous variable

For illustration purposes, we in this example convert the covariate $x_1$ to a continuous variable by explicitly setting it as a continuous variable. The response function is also updated so it processes continuous values for $x_1$.

By making this change, we are no longer solving the same problem as the one in the two cases above. Nonetheless the changes illustrate that convergence is typically achieved faster if not all covariates are integer.

### Defining covariates
Covariates can either be defined via a list of tuples (see Examples 1 - 5) or via a nested dict as show earlier in this example. In the former case, the data type of the covariate is inferred from the data provided to define the covariates (a single float data type in a tuple will cast it as type float).

For completeness we show below how to define the same covariates (one an integer, the other a continuous) using both approaches. 

In [ ]:
# define the range of interest
x0_init = 2
x1_init = 6.0

# using list of tuples
# in this case the covariates will be assigned names "covar0", "covar1"
# covars2d_tuples = [(x0_init, 0 , x0scale), (x1_init, 0, x1scale)]

# create the covariate-defining data structure as a named nested dicts
# with this approach the covariates can be named directly
covars2d_dict = {
    x0_name: {
        "guess": x0_init,
        "min": 0,
        "max": x0scale,
        "type": int,
    },
    x1_name: {
        "guess": x1_init,
        "min": 0,
        "max": x1scale,
        "type": float,  # change data type to continuous
    },
}

Update the response function so it accepts $x_1$ as a continuous variable

In [ ]:
# the function to optimize
# takes a pandas dataframe as input and returns an array
def f_cont(x):
    x0, x1 = np.meshgrid(x["x0"].values, x["x1"].values)
    x0p = np.round(x0,0)/x0scale
    x1p = x1/x1scale  # updated here
    return np.sin(x0p*np.pi)*(-(6*x1p - 2)**2*np.sin(12*x1p-4))

In [ ]:
# helper for plotting
def f_cont_plot(x0vec,x1vec):
    xdf = pd.DataFrame({"x0": x0vec, "x1": x1vec})
    x0p, x1p = np.meshgrid(x0vec, x1vec)
    return x0p, x1p, f_cont(xdf)
    
# generate the data to plot
x0vec = np.linspace(0,x0scale,200)
x1vec = np.linspace(0,x1scale,200)

x0p, x1p, output = f_cont_plot(x0vec, x1vec)

# Set up a figure twice as tall as it is wide
fig = plt.figure(figsize=(12,6)) 

# First subplot: contour plot 
ax = fig.add_subplot(1, 2, 1)
cs = ax.contourf(x0p, x1p, output, cmap="jet")
ax.set_xlabel("$x_0$")
ax.set_ylabel("$x_1$")
cbar = fig.colorbar(cs)


# Second subplot: surface plot
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.view_init(elev=20., azim=19)
ax.plot_surface(x0p, x1p, output, rstride=1, cstride=1,linewidth=1, antialiased=True, shade=True, cmap="jet")
ax.set_xlabel("$x_0$")
ax.set_ylabel("$x_1$")
ax.set_zlabel("Response")

plt.show()

### Solve the problem

In [ ]:
# initialize class instance
cc3 = CreativeProject(covars=covars2d_dict)

# number of iterations
max_iter = 90

# run the auto-method
cc3.auto(response_samp_func=f_cont, max_iter=max_iter)

In [ ]:
# run current_best method
cc3.current_best()

In [ ]:
# plot convergence
cc3.plot_convergence()

In [ ]:
# plot evolution of best result of objective function
cc3.plot_best_objective()